<a href="https://colab.research.google.com/github/notsofun/-/blob/main/Usage_of_deepseek_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [6]:
import re
import torch

# 选择 GPU，如果不可用，则回退到 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

prompt = "你觉得爱上一个和前任很像的人合适吗"

messages = [
    {"role": "system", "content": "你是一个善解人意的小哥哥，可以从心理学角度解答人的困惑。你需要以聊天的口吻，而不是长篇大论。"},
    {"role": "user", "content": prompt}
]

def get_response(message, device):

  model.to(device)

  text = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

  model_inputs = tokenizer([text], return_tensors="pt", padding=True).to(device)

  # pad_token_id 是一个参数，要在函数的里面使用
  generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True, pad_token_id = tokenizer.eos_token_id)

  generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  # 使用正则表达式删除 <think> 到 </think> 之间的内容
  response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)

  # 去掉多余的空格和换行
  response = response.strip()

  return response

response = get_response(messages,device)

print(response)

如果一个人和前任看起来很像，可能会带来很多问题。尽管外表和气质可能相似，但性格和情感表达可能有冲突，导致沟通困难。因此，建议考虑性格和成长轨迹，找到真正互补的人。
